# Chapter 4 Exercises

1. _Which Linear Regression training algorithm can you use if you have a training set with millions of features?_<br>
SGD, esp. batch
1. _Suppose the features in your training set have very different scales. Which algorithms might suffer from this, and how? What can you do about it?_<br>
most of them. will introduce bias
    1. nn: min max scaling. otherwise: standardization (bias removal)
1. _Can Gradient Descent get stuck in a local minimum when training a Logistic Regression model?_<br>
no, it has a convex cost function
1. _Do all Gradient Descent algorithms lead to the same model, provided you let them run long enough?_<br> only if they converge at the global minimum
1. _Suppose you use Batch Gradient Descent and you plot the validation error at every epoch. If you notice that the validation error consistently goes up, what is likely going on? How can you fix this?_<br>
it is overfitting. record the minimum, wait for it to be above for a while, and then roll back [could also be diverging due to too high learning rate if training error is going up too]
1. _Is it a good idea to stop Mini-batch Gradient Descent immediately when the validation error goes up?_<br>
no, you might not have converged to the global minimum yet. record minimum and wait to roll back
1. _Which Gradient Descent algorithm (among those we discussed) will reach the vicinity of the optimal solution the fastest? Which will actually converge? How can you make the others converge as well?_
    1. stochastic
    1. batch, but any if right learning rate, or decreasing learning rate
    1. decrease learning rate in vicinity, and/or roll back
1. _Suppose you are using Polynomial Regression. You plot the learning curves and you notice that there is a large gap between the training error and the validation error. What is happening? What are three ways to solve this?_<br>
overfitting the training data
    1. regularization
    1. reduce complexity of model
    1. more training data
1. _Suppose you are using Ridge Regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter α or reduce it?_<br>
underfitting: bias
    1. reduce it
1. _Why would you want to use:_
    1. _Ridge Regression instead of plain Linear Regression (i.e., without any regularization)?_<br>
    to reduce overfitting, esp. with polynomial feature creation
    1. _Lasso instead of Ridge Regression?_<br>
    implements a sort of feature selection, and helps GD converge
    1. _Elastic Net instead of Lasso?_<br>
    Elastic Net is less eratic than Lasso
    1. (default: Ridge, unless you think only a few features are useful, then Elastic Net)
1. _Suppose you want to classify pictures as outdoor/indoor and daytime/nighttime. Should you implement two Logistic Regression classifiers or one Softmax Regression classifier?_<br>
2 logistic regression, as the two classes are mutually independent

12. _Implement Batch Gradient Descent with early stopping for Softmax Regression (without using Scikit-Learn)._

In [ ]:
from copy import deepcopy

eta = 0.1  # learning rate
n_iterations = 1000
m = 100

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients
    
    
# prepare the data
poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler())
    ])
X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True,
                       penalty=None, learning_rate="constant", eta0=0.0005)

minimum_val_error = float("inf")
best_epoch = None
best_model = None
for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train)  # continues where it left off
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = deepcopy(sgd_reg)

In [84]:
import numpy as np

def softmax_score(x_i, param_matrix, k):
    return np.dot(param_matrix[k], x_i)

def softmax(x_i, param_matrix, k):
    num_classes = param_matrix.shape[0]
    result = np.zeros(num_classes)
    for j in range(num_classes):
        result[j] = np.exp(softmax_score(x_i, param_matrix, j))
    score_sum = np.sum(result)
    return result[k]/score_sum
        
def softmax_grad(X, y, param_matrix, k):
    m = X.shape[0]
    sum = np.zeros(X.shape[1])
    for i in range(m):
        x_i = X[i]
        sum += np.multiply((softmax(x_i, param_matrix, k) - y[i][k]), x_i)
    return np.divide(sum, m)

# TODO vectorize
def softmax_grads(X, y, param_matrix):
    num_classes = param_matrix.shape[0]
    result = np.zeros(param_matrix.shape)
    for k in range(num_classes):
        result[k] = softmax_grad(X, y, param_matrix, k)
    return result

def softmax_predict_proba(X, param_matrix):
    m = X.shape[0]
    n_classes = param_matrix.shape[0]
    result = np.zeros((m, n_classes))
    for i in range(m):
        for k in range(n_classes):
            result[i][k] = softmax(X[i], param_matrix, k)
    return result   

def cross_entropy_cost(predict_proba, y):
    m = y.shape[0]
    k = predict_proba.shape[1]
    sum = 0
    for i in range(m):
        for j in range(k):
           sum += y[i][j] * np.log(predict_proba[i][j])
    return -sum/m

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder

iris = datasets.load_iris()

X = iris["data"][:, (2, 3)]  # petal length, petal width
yc = iris["target"]
#cats = Setosa, Versicolour, and Virginica
print(X.shape)
y = np.zeros((yc.shape[0], 3))
for i, c in enumerate(yc):
    y[i][c] = 1
#onehot = OneHotEncoder()
#y = onehot.fit_transform(y)
#print(y)
#print(y[0])
pp = softmax_predict_proba()

## softmax batch gradient descent

In [78]:
eta = 0.1  # learning rate
n_iterations = 1000
m = 100

theta = np.random.randn(3, 2)  # random initialization

for iteration in range(n_iterations):
    gradients = softmax_grads(X, y, theta)
    theta = theta - eta * gradients
    
theta

array([[ 0.60037788, -4.65053192],
       [-0.57849384,  0.35667315],
       [-1.67749522,  3.75841022]])

## softmax batch gradient descent with early stopping

In [ ]:
minimum_val_error = float("inf")
best_epoch = None
best_model = None
theta = np.random.randn(3, 2)  # random initialization
best_theta = theta.copy()
for epoch in range(1000):
    gradients = softmax_grads(X, y, theta)
    theta = theta - eta * gradients

    y_val_predict = softmax_predict_proba(X, theta)
    val_error = cross_entropy_cost(y_val_predict, y)
    print(epoch, val_error, minimum_val_error)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
#        print(epoch)
        print(y_val_predict)
        best_theta = theta.copy()
theta = best_theta

In [86]:
print(np.argmax(np.dot(theta, [5,2])))
predictions = []
for x_i in X:
    predictions.append(np.argmax(np.dot(theta, x_i)))
print(predictions)
#np.argmax(np.dot(theta, X))

predict_p = []
for x_i in X:
    predict_p.append((np.dot(theta, x_i)))
print(softmax_predict_proba(np.array([[5,2]]), theta))


2
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[[0.00360377 0.20721639 0.78917984]]


In [88]:
theta

array([[ 0.98460335, -5.33448553],
       [-0.15859576, -0.45059651],
       [-1.28642178,  3.03758384]])

## sklearn softmax regression reference values

In [72]:
from sklearn.linear_model import LogisticRegression

softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10)
softmax_reg.fit(X, yc)
print(softmax_reg.predict([[5, 2]]))
softmax_reg.predict_proba([[5, 2]])

[2]


array([[6.38014896e-07, 5.74929995e-02, 9.42506362e-01]])

In [36]:
softmax_reg.coef_

array([[-4.58614563, -2.24129385],
       [ 0.16068263, -2.15860167],
       [ 4.425463  ,  4.39989552]])

In [53]:
smrp = softmax_reg.predict(X)

In [87]:
smrp - predictions

array([ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,
        0,  0,  0,  0, -1,  0, -1,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0, -2,  0,  0,  0,  0,  0,  0,  0,
       -1,  0, -1, -1,  0, -1,  0,  0, -1,  0, -1,  0,  0, -1,  0, -1,  0,
       -1,  0,  0, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  1, -1,
       -1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,
        0,  1,  0,  1, -1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  1,  0,  0,  0,  1,  1,  1,  0,  0,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])